In [59]:
import ollama
import psycopg2

prompt = '''
You are a helpful assistant specialising in PostGRE SQL database.
Answer the questions by providing only the SQL statement that is compatible with the PostGRE database.
The response should contain only the generated SQL statement.
This is the question you are required to answer: 
How many accounts does the repid CCCA has?

Here is the relevant context of the database:
CREATE TABLE Accounts (
    AccountName        varchar(40),
    AccountNumber      char(8) primary key,
    AccountValue       DECIMAL(10, 2),
    LastRebalanceDate   date,
    DaysSinceLastRebalance        integer,
    TotalSecurityDriftValue         DECIMAL(10, 2),
	AccountClass		varchar(20),
	RepID				char(4)
);
'''

response = ollama.generate(model="codellama:13b", prompt=prompt)
sql = response['response']
print(sql)
iIndex = sql.find("SELECT")
end = sql.find(";")
if iIndex != -1 and end != -1:
    sql = sql[iIndex:end]
else:
    sql = 'SELECT 1'

DB_PARAMS = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "password",
    "host": "localhost",
    "port": "5432"
}

conn = psycopg2.connect(**DB_PARAMS)
cur = conn.cursor()
cur.execute(sql)
    
rows = cur.fetchall()
output = "\n\n".join([f"Result: {row[0]}" for row in rows])

print(output)

SELECT COUNT(*) FROM Accounts WHERE RepID = 'CCCA';
Result: 5
